# Evaluation by MRI characteristics

In [2]:

import os
import sys
import argparse
import traceback


import logging
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd
from itertools import combinations, permutations

import cmbnet.utils.utils_plotting as utils_plotting
import cmbnet.utils.utils_evaluation as utils_eval
import ast
from cmbnet.commands.evaluate import load_and_prepare_data

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)


In [3]:
eval_dir = "../../data-misc/evaluations"

l1_dirs = ["Scratch-Pretrained-FineTuned"]  # winner
l2_dirs = ["predict_cmb_valid", "predict_cmb_dou", "predict_cmb_crb"]
l3_dirs = ["valloss"]

(
    df_classification,
    df_detection,
    df_segmentation,
    detection_details,
    segmentation_details,
    cmb_results,
) = utils_eval.load_evaluation_data(
    eval_dir, l1_dirs, l2_dirs, l3_dirs
)


In [4]:
# Flatten the data into a list
flattened_data = []
for key, entries in detection_details.items():
    if not entries:
        continue
    for entry in entries:
        # Combine key tuple and entry dictionary
        combined_entry = dict(zip(['Model', 'Dataset', 'Criteria', 'Location'], key))
        combined_entry.update(entry)
        flattened_data.append(combined_entry)

# Create a DataFrame
df_detection_serieslevel = pd.DataFrame(flattened_data)


In [5]:
# Flatten the data into a list
flattened_data = []
for key, entries in cmb_results.items():
    if not entries:
        continue
    for entry in entries:
        # Combine key tuple and entry dictionary
        combined_entry = dict(zip(['Model', 'Dataset', 'Criteria', 'Location'], key))
        combined_entry.update(entry)
        flattened_data.append(combined_entry)


# Create a DataFrame
df_detection_cmblevel = pd.DataFrame(flattened_data)


In [6]:
gt_radiomics_metadata_csv = "../../data-misc/csv/CMB_radiomics_metadata.csv"
gt_cmb_metadata_csv = "../..//data-misc/csv/CMB_metadata_all.csv"
all_studies_csv = "../../data-misc/csv/ALL_studies.csv"

pred_metadata_df = pd.DataFrame()

for dd in ["dou", "crb", "valid"]:
    (
        all_studies_df,
        GT_metadata,
        GT_metadata_radiomics,
        pred_metadata_dd,
        GT_metadata_all,
    ) = load_and_prepare_data(
        all_studies_csv,
        gt_cmb_metadata_csv,
        gt_radiomics_metadata_csv,
        f"/storage/evo1/jorge/datasets/cmb/evaluations_cmb/Scratch-Pretrained-FineTuned/predict_cmb_{dd}/valloss",
    )
    pred_metadata_dd['Dataset'] = dd
    pred_metadata_df = pd.concat([pred_metadata_df, pred_metadata_dd], axis=0)
    
    

valid_studies = os.listdir("/storage/evo1/jorge/datasets/cmb/cmb_valid/Data")
all_studies_testing = all_studies_df[
    (all_studies_df["seriesUID"].isin(valid_studies))
    | (all_studies_df["Dataset"].isin(["DOU", "CRB"]))
]


GT_metadata_all = GT_metadata_all.drop(
    columns=[
        "firstorder_10Percentile",
        "firstorder_90Percentile",
        "firstorder_Energy",
        "firstorder_Entropy",
        "firstorder_InterquartileRange",
        "firstorder_Kurtosis",
        "firstorder_Maximum",
        "firstorder_MeanAbsoluteDeviation",
        "firstorder_Mean",
        "firstorder_Median",
        "firstorder_Minimum",
        "firstorder_Range",
        "firstorder_RobustMeanAbsoluteDeviation",
        "firstorder_RootMeanSquared",
        "firstorder_Skewness",
        "firstorder_TotalEnergy",
        "firstorder_Uniformity",
        "firstorder_Variance",
    ]
)
pred_metadata_df['type'] = "pred"
GT_metadata_all['type'] = "GT"

/storage/evo1/jorge/MicrobleedNet/cmbnet/commands/evaluate.py:146: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  GT_metadata = pd.read_csv(gt_cmb_metadata_csv)
/storage/evo1/jorge/MicrobleedNet/cmbnet/commands/evaluate.py:146: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  GT_metadata = pd.read_csv(gt_cmb_metadata_csv)
/storage/evo1/jorge/MicrobleedNet/cmbnet/commands/evaluate.py:146: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  GT_metadata = pd.read_csv(gt_cmb_metadata_csv)


In [7]:
pred_metadata_df[pred_metadata_df["Dataset"] == "crb"]

,seriesUID,pred_CM,n_voxels,OverlapCMCounts,DistancesToAllCMs,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,shape_Sphericity,shape_SurfaceArea,shape_SurfaceVolumeRatio,shape_VoxelVolume,firstorder_10Percentile,firstorder_90Percentile,firstorder_Energy,firstorder_Entropy,firstorder_InterquartileRange,firstorder_Kurtosis,firstorder_Maximum,firstorder_MeanAbsoluteDeviation,firstorder_Mean,firstorder_Median,firstorder_Minimum,firstorder_Range,firstorder_RobustMeanAbsoluteDeviation,firstorder_RootMeanSquared,firstorder_Skewness,firstorder_TotalEnergy,firstorder_Uniformity,firstorder_Variance,count_dict,com_label,matched_GT_OverlapCMCounts,matched_GT_DistancesToAllCMs,radius,name_counts_dict,percentages_dict,max_key,max_brain_key,max_label,max_brain_label,likely_brain_key,likely_brain_label,BOMBS_label,Dataset,type
0,CRB-1.2.826.1.3680043.9.5282.150415.24477.2447722188652-CMB-2,"(107, 220, 172)",83,"{(107, 221, 171): 83}","{(107, 221, 171): 0.7587013987430363, (111, 221, 135): 18.798159711401173}",0.785924,0.640446,1.936979,3.024423,3.354102,2.692582,3.162278,3.391165,9.718750,2.376967,0.916561,24.028660,2.472402,10.375,122.262395,144.472000,1.501494e+06,9.664538e-01,10.980255,2.892215,157.245407,7.043374,134.210998,134.142273,117.704712,39.540695,4.782389,134.500155,0.283273,1.876868e+05,0.626651,77.699733,{13: 83},13.0,"(107, 221, 171)","(107, 221, 171)",1.353006,{'left pallidum': 83},{13: 1.0},13,13,left pallidum,left pallidum,13,left pallidum,Basal ganglia grey matter,crb,pred
1,CRB-42a2a5865709f9ddc47da60a1556f09552586685-CMB-6,"(89, 192, 170)",87,"{(88, 191, 171): 52}","{(88, 191, 171): 1.0771225055952176, (159, 84, 154): 64.80538387432757, (191, 186, 162): 50.95661742657892, (218, 208, 142): 66.2042239842129, (222, 128, 179): 73.70180574384561, (247, 186, 144): 79.84483420493713}",0.674480,0.338292,1.392383,4.115925,4.527693,3.500000,4.609772,4.609772,10.062500,2.776108,0.757989,29.736617,2.955192,10.875,481.297137,673.425745,2.947371e+07,3.439148e+00,127.747803,2.608523,798.271118,65.527190,576.873847,565.180908,443.483551,354.787567,50.231263,582.046647,0.374204,3.684214e+06,0.103448,5994.863956,"{2: 14, 24: 22}",24.0,"(88, 191, 171)","(88, 191, 171)",1.374401,"{'left cerebral white matter': 14, 'left cerebral cortex': 51, 'CSF': 22}","{2: 0.16091954022988506, 3: 0.5862068965517241, 24: 0.25287356321839083}",3,3,left cerebral cortex,left cerebral cortex,3,left cerebral cortex,Cortex / grey-white junction,crb,pred
2,CRB-1.2.826.1.3680043.9.5282.150415.178.178232327158-CMB-1,"(120, 207, 145)",50,{},"{(181, 292, 177): 54.09398210522128}",0.803017,0.593761,1.554901,2.618731,2.915476,2.236068,2.692582,3.082207,5.671875,2.102887,0.884158,17.395566,3.066987,6.250,135.536957,156.928354,1.082836e+06,9.709506e-01,10.103668,2.356279,164.972214,6.549326,146.949280,148.063553,132.051590,32.920624,4.860305,147.162198,-0.068958,1.353545e+05,0.520000,62.621694,"{2: 24, 13: 26}",2.0,None,None,1.142695,"{'left cerebral white matter': 24, 'left pallidum': 26}","{2: 0.48, 13: 0.52}",13,13,left pallidum,left pallidum,13,left pallidum,Basal ganglia grey matter,crb,pred
3,CRB-1.2.826.1.3680043.9.5282.150415.178.178232327158-CMB-1,"(182, 291, 177)",361,"{(181, 292, 177): 312}","{(181, 292, 177): 1.0062057579046306}",0.633282,0.613315,3.336937,5.440820,6.500000,4.123106,5.590170,6.576473,43.968750,3.445576,0.868913,69.332567,1.576860,45.125,63.286179,122.999176,3.284978e+06,1.913711e+00,29.757256,3.035325,165.351318,17.932501,92.657205,90.950951,44.295372,121.055946,12.377318,95.392149,0.449563,4.106223e+05,0.303597,514.304458,"{41: 298, 42: 63}",41.0,"(181, 292, 177)","(181, 292, 177)",2.208562,"{'right cerebral white matter': 298, 'right cerebral cortex': 63}","{41: 0.8254847645429363, 42: 0.1745152354570637}",41,41,right cerebral white matter,right cerebral white matter

# Undestand data better

In [8]:
df_detection_cmblevel

,Model,Dataset,Criteria,seriesUID,CM,call,type,matched_CM
0,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00001-CMB-2,"(75, 226, 190)",FN,GT,None
1,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00001-CMB-2,"(219, 205, 143)",FN,GT,None
2,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00008-CMB-1,"(179, 110, 241)",FP,pred,None
3,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00008-CMB-1,"(212, 247, 212)",TP,pred,"(212, 247, 211)"
4,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00008-CMB-1,"(212, 247, 211)",TP,GT,"(212, 247, 212)"
...,...,...,...,...,...,...,...,...
547,Scratch-Pretrained-FineTuned,crb,valloss,CRB-42a2a5865709f9ddc47da60a1556f09552586685-CMB-6,"(191, 186, 162)",FN,GT,None
548,Scratch-Pretrained-FineTuned,crb,valloss,CRB-42a2a5865709f9ddc47da60a1556f09552586685-CMB-6,"(218, 208, 142)",FN,GT,None
549,Scratch-Pretrained-FineTuned,crb,valloss,CRB-42a2a5865709f9ddc47da60a1556f09552586685-CMB-6,"(222, 128, 179)",FN,GT,None
550,Scratch-Pretrained-FineTuned,crb,valloss,CRB-42a2a5865709f9ddc47da60a1556f09552586685-CMB-6,"(247, 186, 144)",FN,GT,None


In [9]:
pred_metadata_df['CM'] = pred_metadata_df['pred_CM']

In [10]:

df_detection_cmblevel = df_detection_cmblevel[df_detection_cmblevel["seriesUID"].isin(valid_studies) | df_detection_cmblevel["Dataset"].isin(['dou', 'crb'])]
all_cmbs = pd.concat([GT_metadata_all,pred_metadata_df ]).drop(columns=["Dataset", "Location"])

df_detection_cmblevel = pd.merge(df_detection_cmblevel , 
                                 all_cmbs, 
                                 on=["seriesUID", "CM", "type"], how="left")


In [11]:
df_detection_cmblevel

,Model,Dataset,Criteria,seriesUID,CM,call,type,matched_CM,cmb_id,size,radius,processed_id,RB_label,Multiple,<5mm,Cause,Uncertain,Other,seq_type,res_level,field_strength,TE,subject,patientUID,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,shape_Sphericity,shape_SurfaceArea,shape_SurfaceVolumeRatio,shape_VoxelVolume,count_dict,com_label,name_counts_dict,percentages_dict,max_key,max_brain_key,max_label,max_brain_label,likely_brain_key,likely_brain_label,BOMBS_label,pred_CM,n_voxels,OverlapCMCounts,DistancesToAllCMs,firstorder_10Percentile,firstorder_90Percentile,firstorder_Energy,firstorder_Entropy,firstorder_InterquartileRange,firstorder_Kurtosis,firstorder_Maximum,firstorder_MeanAbsoluteDeviation,firstorder_Mean,firstorder_Median,firstorder_Minimum,firstorder_Range,firstorder_RobustMeanAbsoluteDeviation,firstorder_RootMeanSquared,firstorder_Skewness,firstorder_TotalEnergy,firstorder_Uniformity,firstorder_Variance,matched_GT_OverlapCMCounts,matched_GT_DistancesToAllCMs
0,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00001-CMB-2,"(75, 226, 190)",FN,GT,None,0.0,5889.0,11.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SWI,high,1.5/3,NaN,00001,00001,0.557328,0.517016,7.953666,15.383797,16.347783,9.861541,16.620770,16.650826,732.364583,8.573826,0.733265,535.847765,0.731668,736.125,"{2: 516, 24: 405}",24.0,"{'left cerebral white matter': 516, 'left cerebral cortex': 4968, 'CSF': 405}","{2: 0.08762098828323994, 3: 0.8436067244014264, 24: 0.06877228731533368}",3,3,left cerebral cortex,left cerebral cortex,2,left cerebral white matter,Subcortical white matter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00001-CMB-2,"(219, 205, 143)",FN,GT,None,1.0,15.0,1.530000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SWI,high,1.5/3,NaN,00001,00001,0.194328,0.000000,0.000000,4.787597,4.500000,1.000000,4.527693,4.527693,1.239583,0.930365,0.601802,9.272880,7.480642,1.875,{41: 15},41.0,{'right cerebral white matter': 15},{41: 1.0},41,41,right cerebral white matter,right cerebral white matter,41,right cerebral white matter,Subcortical white matter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00008-CMB-1,"(179, 110, 241)",FP,pred,None,NaN,NaN,1.004847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.404323,0.311907,1.115098,3.575093,4.031129,1.581139,4.000000,4.031129,3.635417,1.445494,0.766975,14.907522,4.100636,4.250,{42: 34},42.0,{'right cerebral cortex': 34},{42: 1.0},42,42,right cerebral cortex,right cerebral cortex,42,right cerebral cortex,Cortex / grey-white junction,"(179, 110, 241)",34.0,{},"{(212, 247, 211): 71.94534431917157}",9518.326172,35940.351953,1.992363e+10,5.028639,19082.241455,2.169568,46406.972656,9688.555433,21426.770580,18475.508789,5743.824707,40663.147949,7176.561209,24207.209762,0.551002,2.490453e+09,0.031142,1.268825e+08,None,None
3,Scratch-Pretrained-FineTuned,valid,valloss,RODEJA-00008-CMB-1,"(212, 247, 212)",TP,pred,"(212, 247, 211)",NaN,NaN,1.227640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.881651,0.498912,1.492084,2.990673,3.605551,3.041381,3.535534,3.674235,7.005208,2.636730,0.787558,22.480988,3.209182,7.750,"{24: 16, 41: 33, 42: 13}",24.0,"{'CSF': 16, 'right cerebral white matter': 33, 'right cerebral cortex': 13}","{24: 0.25806451612903225, 41: 0.532258064516129, 42: 0.20967741935483872}",41,41,right cerebral white matter,right cerebral white matter,41,right cerebral white matter,Subcortical white matter,"(212, 247, 212)",62.0,"{(212, 247, 211): 51}","{(212, 247, 211): 0.6632543611274814}",24530.958594,85917.876562,1.711057e+11,5.921938,24380.729492,2.844671,100393.523438,16526.853255,48251.938036,44530.257812,20509.607422,79883.916016,10840.849223,525

In [12]:
GT_metadata_all.groupby

<bound method DataFrame.groupby of                                                              seriesUID  \
0                                                   RODEJA-00058-CMB-2   
1                                                   RODEJA-00058-CMB-2   
2                                                  MOMENI-238_T1-CMB-3   
3                                                  MOMENI-238_T1-CMB-3   
4                                                  MOMENI-238_T1-CMB-3   
..                                                                 ...   
952        CRB-1.2.826.1.3680043.9.5282.150415.650.650232327158-CMB-13   
953  CRB-1.3.12.2.1107.5.2.6.24111.30000019082104124198400000043-CMB-4   
954  CRB-1.3.12.2.1107.5.2.6.24111.30000019082104124198400000043-CMB-4   
955  CRB-1.3.12.2.1107.5.2.6.24111.30000019082104124198400000043-CMB-4   
956  CRB-1.3.12.2.1107.5.2.6.24111.30000019082104124198400000043-CMB-4   

     cmb_id               CM  size  radius  processed_id  RB_label  \
0     

In [13]:
df_detection_cmblevel.columns

Index(['Model', 'Dataset', 'Criteria', 'seriesUID', 'CM', 'call', 'type',
       'matched_CM', 'cmb_id', 'size', 'radius', 'processed_id', 'RB_label',
       'Multiple', '<5mm', 'Cause', 'Uncertain', 'Other', 'seq_type',
       'res_level', 'field_strength', 'TE', 'subject', 'patientUID',
       'shape_Elongation', 'shape_Flatness', 'shape_LeastAxisLength',
       'shape_MajorAxisLength', 'shape_Maximum2DDiameterColumn',
       'shape_Maximum2DDiameterRow', 'shape_Maximum2DDiameterSlice',
       'shape_Maximum3DDiameter', 'shape_MeshVolume', 'shape_MinorAxisLength',
       'shape_Sphericity', 'shape_SurfaceArea', 'shape_SurfaceVolumeRatio',
       'shape_VoxelVolume', 'count_dict', 'com_label', 'name_counts_dict',
       'percentages_dict', 'max_key', 'max_brain_key', 'max_label',
       'max_brain_label', 'likely_brain_key', 'likely_brain_label',
       'BOMBS_label', 'pred_CM', 'n_voxels', 'OverlapCMCounts',
       'DistancesToAllCMs', 'firstorder_10Percentile',
       'firstorder_90

In [14]:
colsss = [
    'shape_Elongation', 'shape_Flatness', 
    'shape_Maximum3DDiameter', 'shape_MeshVolume', 
    'shape_Sphericity',
]

# Building the aggregation dictionary
agg_dict = {col: ['mean', 'std'] for col in colsss}
agg_dict.update({"seriesUID": ['count']})

# Apply aggregation
grouped_stats = df_detection_cmblevel.groupby(['call', 'type', 'Dataset']).agg(agg_dict)

# The grouped_stats DataFrame will now have the mean, std, and count for each specified column, grouped by 'call'


In [15]:
grouped_stats

shape_Elongation           shape_Flatness            \
                              mean       std           mean       std   
call type Dataset                                                       
FN   GT   crb             0.581624  0.251435       0.502844  0.203919   
          dou             0.878068  0.112492       0.777708  0.149967   
          valid           0.578283  0.239423       0.433424  0.249812   
FP   pred crb             0.723720  0.176896       0.520254  0.280420   
          dou             0.703330  0.117182       0.478932  0.248351   
          valid           0.680654  0.180909       0.555878  0.177197   
TP   GT   crb             0.697788  0.246448       0.590249  0.180857   
          dou             0.853560  0.105394       0.766509  0.116329   
          valid           0.598698  0.221165       0.487293  0.171903   
     pred crb             0.764110  0.128769       0.643933  0.208438   
          dou             0.826668  0.099888       0.674807  0.152198   
          valid           0.672778  0.192653       0.555129  0.208820   

                  shape_Maximum3DDiameter           shape_MeshVolume  \
                                     mean       std             mean   
call type Dataset                                                      
FN   GT   crb                    6.104597  2.067929        30.881730   
          dou                    3.712472  0.965392        15.463315   
          valid                  5.749997  4.352590        45.571915   
FP   pred crb                    2.773443  0.834922         4.768229   
          dou                    2.423387  0.599710         2.983724   
          valid                  3.316892  0.943040         5.867560   
TP   GT   crb                    5.387636  1.526687        25.721354   
          dou                    3.792398  0.674593        15.450572   
          valid                  5.228463  2.091814        16.304613   
     pred crb                    3.799266  1.332565        14.725366   
          dou                    3.323984  0.770524         9.208116   
          valid                  3.964927  1.835501        11.749193   

                              shape_Sphericity           seriesUID  
                          std             mean       std     count  
call type Dataset                                                   
FN   GT   crb       29.678145         0.775411  0.059102        89  
          dou       10.822421         0.854008  0.021241        23  
          valid    144.409311         0.739305  0.111009        26  
FP   pred crb        4.294734         0.816192  0.077033        16  
          dou        2.982532         0.820752  0.059568        10  
          valid      4.165096         0.824815  0.061053        68  
TP   GT   crb       27.384456         0.778819  0.045708        38  
          dou       14.134176         0.842732  0.028893        51  
          valid     18.365939         0.769184  0.067008        71  
     pred crb       14.949151         0.854190  0.041395        38  
          dou        6.133131         0.855476  0.047317        51  
          valid     14.068444         0.837740  0.064507        71

In [16]:
# Assuming 'grouped_stats' is your DataFrame from the previous aggregation
# Format the DataFrame for "mean ± std"
for col in colsss:
    grouped_stats[col, 'mean±std'] = grouped_stats[col, 'mean'].map('{:.2f}'.format) + ' ± ' + grouped_stats[col, 'std'].map('{:.2f}'.format)

# Drop the original mean and std columns
grouped_stats = grouped_stats.drop([(col, 'mean') for col in colsss] + [(col, 'std') for col in colsss], axis=1)

# Flatten the MultiIndex columns
grouped_stats.columns = [' '.join(col).strip() if 'mean' not in col else col[0] for col in grouped_stats.columns.values]

# Convert to LaTeX
latex_table = grouped_stats.to_latex(escape=False, index=True, multirow=True, multicolumn=True, column_format='ll' + 'c' * (len(colsss) + 1))  # Adjust the column format as needed

print(latex_table)


\begin{tabular}{llcccccc}
\toprule
 &  &  & seriesUID count & shape_Elongation mean±std & shape_Flatness mean±std & shape_Maximum3DDiameter mean±std & shape_MeshVolume mean±std & shape_Sphericity mean±std \\
call & type & Dataset &  &  &  &  &  &  \\
\midrule
\multirow[t]{3}{*}{FN} & \multirow[t]{3}{*}{GT} & crb & 89 & 0.58 ± 0.25 & 0.50 ± 0.20 & 6.10 ± 2.07 & 30.88 ± 29.68 & 0.78 ± 0.06 \\
 &  & dou & 23 & 0.88 ± 0.11 & 0.78 ± 0.15 & 3.71 ± 0.97 & 15.46 ± 10.82 & 0.85 ± 0.02 \\
 &  & valid & 26 & 0.58 ± 0.24 & 0.43 ± 0.25 & 5.75 ± 4.35 & 45.57 ± 144.41 & 0.74 ± 0.11 \\
\cline{1-9} \cline{2-9}
\multirow[t]{3}{*}{FP} & \multirow[t]{3}{*}{pred} & crb & 16 & 0.72 ± 0.18 & 0.52 ± 0.28 & 2.77 ± 0.83 & 4.77 ± 4.29 & 0.82 ± 0.08 \\
 &  & dou & 10 & 0.70 ± 0.12 & 0.48 ± 0.25 & 2.42 ± 0.60 & 2.98 ± 2.98 & 0.82 ± 0.06 \\
 &  & valid & 68 & 0.68 ± 0.18 & 0.56 ± 0.18 & 3.32 ± 0.94 & 5.87 ± 4.17 & 0.82 ± 0.06 \\
\cline{1-9} \cline{2-9}
\multirow[t]{6}{*}{TP} & \multirow[t]{3}{*}{GT} & crb & 38 & 0.

In [23]:
GT_metadata_all[
    GT_metadata_all['CM'] == (61, 70, 134)
    
]
GT_metadata_all[
    GT_metadata_all['CM'] == (188, 240, 93)
    
]

,seriesUID,cmb_id,CM,size,radius,processed_id,RB_label,Location,Multiple,<5mm,Cause,Uncertain,Other,seq_type,res_level,field_strength,TE,subject,patientUID,Dataset,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,shape_Sphericity,shape_SurfaceArea,shape_SurfaceVolumeRatio,shape_VoxelVolume,count_dict,com_label,name_counts_dict,percentages_dict,max_key,max_brain_key,max_label,max_brain_label,likely_brain_key,likely_brain_label,BOMBS_label,type
948,CRB-1.2.826.1.3680043.9.5282.150415.650.650232327158-CMB-13,8,"(188, 240, 93)",96,2.84,8.0,1.0,Cortex / grey-white junction,True,True,Other,NaN,NaN,T2S,high,1.5/3,32.5,1.2.826.1.3680043.9.5282.150415.650.650232327158,1.2.826.1.3680043.9.5282.150415.650.650232327158,CRB,0.245844,0.229113,1.521858,6.642398,6.082763,1.802776,6.082763,6.164414,10.989583,1.632993,0.730505,32.722627,2.977604,12.0,{24: 8},42.0,"{'CSF': 8, 'right cerebral cortex': 88}","{24: 0.08333333333333333, 42: 0.9166666666666666}",42,42,right cerebral cortex,right cerebral cortex,42,right cerebral cortex,Cortex / grey-white junction,GT
